# [Predictive Clinical Neuroscience Toolkit](https://github.com/amarquand/PCNtoolkit) 
# Normative Modeling Tutorial Using Multi-Site Cortical Thickness Data
# Part 2: Run the normative models

### Created by [Saige Rutherford](https://twitter.com/being_saige) 

<div>
<img src="../../data/NormModelSetup.png" width="500"/>
</div>

### Preparation

In [ ]:
# re-run if you did not run part 1 notebook
! git clone https://github.com/predictive-clinical-neuroscience/PCNtoolkit-demo.git

In [ ]:
# re-run if you did not run part 1 notebook
! cd PCNtoolkit-demo/

In [ ]:
# re-run if you did not run part 1 notebook
! pip install -r requirements.txt

# Step 5: Run normative model

In [5]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import joypy
import matplotlib.pyplot as plt

from pcntoolkit.normative import estimate, evaluate
from pcntoolkit.utils import create_bspline_basis, compute_MSLL

In [6]:
# set this path to wherever your ROI_models folder is located (where you copied all of the covariate & response text files to in Part 1)
data_dir = 'data/ROI_models/'

In [7]:
# Create a list of all the ROIs you want to run a normative model for
roi_ids = ['lh_MeanThickness_thickness',
           'rh_MeanThickness_thickness',
           'lh_bankssts_thickness',
           'lh_caudalanteriorcingulate_thickness',
           'lh_superiorfrontal_thickness',
           'rh_superiorfrontal_thickness']

When we split the data into train and test sets, we did not reset the index. This means that the row numbers in the train/test matrices are still the same as before splitting the data. We will need the test set row numbers of which subjects belong to which site in order to evaluate per site performance metrics, so we need to reset the row numbers in the train/test split matrices.

In [8]:
x_col_names = ['age', 'sex', 'site_cam', 'site_hcp', 'site_ixi']
X_train = pd.read_csv('data/covariate_files/cov_tr.txt', sep='\t', header=None, names=x_col_names)
X_test = pd.read_csv('data/covariate_files/cov_te.txt', sep='\t', header=None, names=x_col_names)
y_train = pd.read_csv('data/response_files/resp_tr.txt', sep='\t', header=None)
y_test = pd.read_csv('data/response_files/resp_te.txt', sep='\t', header=None)

In [9]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

### Extract site indices

Get site ids so that we can evaluate the test metrics independently for each site

In [10]:
cam_idx = X_test.index[X_test['site_cam' ]== 1].to_list()
hcp_idx = X_test.index[X_test['site_hcp'] == 1].to_list()
ixi_idx = X_test.index[X_test['site_ixi'] == 1].to_list()

# Save the site indices into a single list
sites = [cam_idx, hcp_idx, ixi_idx]

# Create a list with sites names to use in evaluating per-site metrics
site_names = ['cam', 'hcp', 'ixi']

### Basis expansion

Now, we set up a B-spline basis set that allows us to perform nonlinear regression using a linear model. This basis is deliberately chosen to not to be too flexible so that in can only model relatively slowly varying trends. To increase the flexibility of the model you can change the parameterisation (e.g. by adding knot points to the Bspline basis or increasing the order of the interpolating polynomial). 

Note that in the neuroimaging literature, it is more common to use a polynomial basis expansion for this. Piecewise polynomials like B-splines are superior because they do not introduce a global curvature. See the reference below for further information.

[Primer on regression splines](https://cran.r-project.org/web/packages/crs/vignettes/spline_primer.pdf)

[Reference for why polynomials are a bad idea](https://www.sciencedirect.com/science/article/abs/pii/S1053811910000832?via%3Dihub)


In [11]:
# Create a cubic B-spline basis (used for regression)
xmin = 10#16 # xmin & xmax are the boundaries for ages of participants in the dataset
xmax = 95#90
B = create_bspline_basis(xmin, xmax)

# create the basis expansion for the covariates for each of the 
for roi in roi_ids: 
    print('Creating basis expansion for ROI:', roi)
    roi_dir = os.path.join(data_dir, roi)
    os.chdir(roi_dir)
    
    # create output dir 
    os.makedirs(os.path.join(roi_dir,'blr'), exist_ok=True)
    
    # load train & test covariate data matrices
    X_tr = np.loadtxt(os.path.join(roi_dir, 'cov_tr.txt'))
    X_te = np.loadtxt(os.path.join(roi_dir, 'cov_te.txt'))

    # add intercept column 
    X_tr = np.concatenate((X_tr, np.ones((X_tr.shape[0],1))), axis=1)
    X_te = np.concatenate((X_te, np.ones((X_te.shape[0],1))), axis=1)
    np.savetxt(os.path.join(roi_dir, 'cov_int_tr.txt'), X_tr)
    np.savetxt(os.path.join(roi_dir, 'cov_int_te.txt'), X_te)
    
    # create Bspline basis set 
    Phi = np.array([B(i) for i in X_tr[:,0]])
    Phis = np.array([B(i) for i in X_te[:,0]])
    X_tr = np.concatenate((X_tr, Phi), axis=1)
    X_te = np.concatenate((X_te, Phis), axis=1)
    np.savetxt(os.path.join(roi_dir, 'cov_bspline_tr.txt'), X_tr)
    np.savetxt(os.path.join(roi_dir, 'cov_bspline_te.txt'), X_te)

Creating basis expansion for ROI: lh_MeanThickness_thickness
Creating basis expansion for ROI: rh_MeanThickness_thickness
Creating basis expansion for ROI: lh_bankssts_thickness
Creating basis expansion for ROI: lh_caudalanteriorcingulate_thickness
Creating basis expansion for ROI: lh_superiorfrontal_thickness
Creating basis expansion for ROI: rh_superiorfrontal_thickness


### Prepare output structures

In [12]:
# Create pandas dataframes with header names to save out the overall and per-site model evaluation metrics
blr_metrics = pd.DataFrame(columns = ['ROI', 'MSLL', 'EV', 'SMSE', 'RMSE', 'Rho'])
blr_site_metrics = pd.DataFrame(columns = ['ROI', 'site', 'y_mean', 'y_var', 'yhat_mean', 'yhat_var', 'MSLL', 'EV', 'SMSE', 'RMSE', 'Rho'])

### Estimate the normative models

In this step, we estimate the normative models one at a time. In principle we could also do this on the whole data matrix at once (e.g. with the response variables stored in a n_subjects x n_brain_measures numpy array). However, doing it this way gives us some extra flexibility in that it does not require that the subjects are exactly the same for each of the brain measures. 

This code fragment will loop through each region of interest in the roi_ids list (set a few code blocks above) using Bayesian linear regression and evaluate the model on the independent test set. It will then compute error metrics such as the explained variance, mean standardized log loss and Pearson correlation between true and predicted test responses separately for each scanning site. 

We supply the estimate function with a few specific arguments that are worthy of commenting on: 
* alg = 'blr' : specifies we should use Bayesian linear regression
* optimizer = 'powell' : use Powell's derivative-free optimization method (faster in this case than L-BFGS)
* savemodel = False : do not write out the final estimated model to disk
* saveoutput = False : return the outputs directly rather than writing them to disk
* standardize = False : Do not standardize the covariates or response variables

One important consideration is whether or not to standardize. Whilst this generally only has a minor effect on the final model accuracy, it has implications for the interpretation of models and how they are configured. If the covariates and responses are both standardized, the model will return standardized coefficients. If (as in this case) the response variables are not standardized, then the scaling both covariates and responses will be reflected in the estimated coefficients. Also, under the linear modelling approach employed here, if the coefficients are unstandardized and do not have a zero mean, it is necessary to add an intercept column to the design matrix. This is done in the code block above. 

In [ ]:
# Loop through ROIs
for roi in roi_ids: 
    print('Running ROI:', roi)
    roi_dir = os.path.join(data_dir, roi)
    os.chdir(roi_dir)
     
    # configure the covariates to use. Change *_bspline_* to *_int_* to 
    cov_file_tr = os.path.join(roi_dir, 'cov_bspline_tr.txt')
    cov_file_te = os.path.join(roi_dir, 'cov_bspline_te.txt')
    
    # load train & test response files
    resp_file_tr = os.path.join(roi_dir, 'resp_tr.txt')
    resp_file_te = os.path.join(roi_dir, 'resp_te.txt') 
    
    # run a basic model
    yhat_te, s2_te, nm, Z, metrics_te = estimate(cov_file_tr, 
                                                 resp_file_tr, 
                                                 testresp=resp_file_te, 
                                                 testcov=cov_file_te, 
                                                 alg = 'blr', 
                                                 optimizer = 'powell', 
                                                 savemodel = False, 
                                                 saveoutput = False,
                                                 standardize = False)
    # display and save metrics
    print('EV=', metrics_te['EXPV'][0])
    print('RHO=', metrics_te['Rho'][0])
    print('MSLL=', metrics_te['MSLL'][0])
    blr_metrics.loc[len(blr_metrics)] = [roi, metrics_te['MSLL'][0], metrics_te['EXPV'][0], metrics_te['SMSE'][0], 
                                         metrics_te['RMSE'][0], metrics_te['Rho'][0]]
    
    # Compute metrics per site in test set, save to pandas df
    # load true test data
    X_te = np.loadtxt(cov_file_te)
    y_te = np.loadtxt(resp_file_te)
    y_te = y_te[:, np.newaxis] # make sure it is a 2-d array
    
    # load training data (required to compute the MSLL)
    y_tr = np.loadtxt(resp_file_tr)
    y_tr = y_tr[:, np.newaxis]
    
    for num, site in enumerate(sites):     
        y_mean_te_site = np.array([[np.mean(y_te[site])]])
        y_var_te_site = np.array([[np.var(y_te[site])]])
        yhat_mean_te_site = np.array([[np.mean(yhat_te[site])]])
        yhat_var_te_site = np.array([[np.var(yhat_te[site])]])
        
        metrics_te_site = evaluate(y_te[site], yhat_te[site], s2_te[site], y_mean_te_site, y_var_te_site)
        
        site_name = site_names[num]
        blr_site_metrics.loc[len(blr_site_metrics)] = [roi, site_names[num],
                                                       y_mean_te_site[0],
                                                       y_var_te_site[0],
                                                       yhat_mean_te_site[0],
                                                       yhat_var_te_site[0],
                                                       metrics_te_site['MSLL'][0],
                                                       metrics_te_site['EXPV'][0],
                                                       metrics_te_site['SMSE'][0],
                                                       metrics_te_site['RMSE'][0],
                                                       metrics_te_site['Rho'][0]]

In [14]:
os.chdir(data_dir)

In [15]:
# Save per site test set metrics variable to CSV file
blr_site_metrics.to_csv('blr_site_metrics.csv', index=False, index_label=None)

In [16]:
# Save overall test set metrics to CSV file
blr_metrics.to_csv('blr_metrics.csv', index=False, index_label=None)

# Step 6: Interpreting model performance

Output evaluation metrics definitions: 
* yhat - predictive mean
* ys2 - predictive variance
* nm - normative model
* Z - deviance scores
* Rho - Pearson correlation between true and predicted responses
* pRho - parametric p-value for this correlation
* RMSE - root mean squared error between true/predicted responses
* SMSE - standardised mean squared error
* EV - explained variance
* MSLL - mean standardized log loss
    * See page 23 in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf